In [5]:
import os
os.chdir("../")

In [6]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [7]:
resultDir = ' '
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small','medium','big']
models = ['gcnn:Lodi','ml-competitor:extratrees_gcnn_agg','ml-competitor:lambdamart_khalil','gcnn:GraphConv'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [8]:
df1 = pd.read_csv(f'')
df = pd.concat([df1],ignore_index=True)
df

,policy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:GraphConv,0,small,data/instances/indset/transfer_500_4/instance_...,35,109,6.38,0.000000,optimal,0,0,6.829113,6.750000
1,gcnn:GraphConv,1,small,data/instances/indset/transfer_500_4/instance_...,83,159,6.99,0.000000,optimal,0,0,7.235511,7.203125
2,gcnn:GraphConv,2,small,data/instances/indset/transfer_500_4/instance_...,87,146,7.07,0.000000,optimal,0,0,7.166723,7.125000
3,gcnn:GraphConv,3,small,data/instances/indset/transfer_500_4/instance_...,119,224,8.73,0.000000,optimal,0,0,8.971030,8.937500
4,gcnn:GraphConv,4,small,data/instances/indset/transfer_500_4/instance_...,55,117,6.60,0.000000,optimal,0,0,6.705128,6.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,ml-competitor:lambdamart_khalil,0,big,data/instances/indset/transfer_1500_4/instance...,109296,142666,7200.00,0.019385,timelimit,0,0,7211.400619,7209.718750
1496,ml-competitor:lambdamart_khalil,1,big,data/instances/indset/transfer_1500_4/instance...,181601,199394,7200.01,0.020801,timelimit,0,0,7215.462562,7213.937500
1497,ml-competitor:lambdamart_khalil,2,big,data/instances/indset/transfer_1500_4/instance...,156873,186054,7200.00,0.021325,timelimit,0,0,7214.032579,7211.531250
1498,ml-competitor:lambdamart_khalil,3,big,data/instances/indset/transfer_1500_4/instance...,242314,238948,7200.00,0.019312,timelimit,0,0,7216.955102,7213.812500


In [9]:
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})
df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()
df_gcnns

,policy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:GraphConv,0,small,data/instances/indset/transfer_500_4/instance_...,35.0,109.0,6.38,0.000000,optimal,0,0,6.829113,6.750000
1,gcnn:GraphConv,1,small,data/instances/indset/transfer_500_4/instance_...,83.0,159.0,6.99,0.000000,optimal,0,0,7.235511,7.203125
2,gcnn:GraphConv,2,small,data/instances/indset/transfer_500_4/instance_...,87.0,146.0,7.07,0.000000,optimal,0,0,7.166723,7.125000
3,gcnn:GraphConv,3,small,data/instances/indset/transfer_500_4/instance_...,119.0,224.0,8.73,0.000000,optimal,0,0,8.971030,8.937500
4,gcnn:GraphConv,4,small,data/instances/indset/transfer_500_4/instance_...,55.0,117.0,6.60,0.000000,optimal,0,0,6.705128,6.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,ml-competitor:lambdamart_khalil,0,big,data/instances/indset/transfer_1500_4/instance...,109296.0,142666.0,7200.00,0.019385,timelimit,0,0,7211.400619,7209.718750
1496,ml-competitor:lambdamart_khalil,1,big,data/instances/indset/transfer_1500_4/instance...,181601.0,199394.0,7200.01,0.020801,timelimit,0,0,7215.462562,7213.937500
1497,ml-competitor:lambdamart_khalil,2,big,data/instances/indset/transfer_1500_4/instance...,156873.0,186054.0,7200.00,0.021325,timelimit,0,0,7214.032579,7211.531250
1498,ml-competitor:lambdamart_khalil,3,big,data/instances/indset/transfer_1500_4/instance...,242314.0,238948.0,7200.00,0.019312,timelimit,0,0,7216.955102,7213.812500


In [10]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [11]:
dfgcnns_gmean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes        stime
type   policy                                                       
big    gcnn:GraphConv                     104270.890116  4303.176479
       gcnn:Lodi                          136951.083542  5453.099332
       ml-competitor:extratrees_gcnn_agg  103607.998748  7037.980723
       ml-competitor:lambdamart_khalil    140796.509738  5793.664749
medium gcnn:GraphConv                       5557.093286   254.646785
       gcnn:Lodi                           41537.382519  1271.874807
       ml-competitor:extratrees_gcnn_agg   80291.347430  4063.897183
       ml-competitor:lambdamart_khalil     25133.600183   648.536246
small  gcnn:GraphConv                         20.931217     5.761901
       gcnn:Lodi                             103.113082    15.824029
       ml-competitor:extratrees_gcnn_agg      57.118387    11.532615
       ml-competitor:lambdamart_khalil        46.444695     6.483165

In [12]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'gcnn:Lodi')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [13]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   policy                                               
big    gcnn:GraphConv                     0.761373  0.789125
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  0.756533  1.290639
       ml-competitor:lambdamart_khalil    1.028079  1.062454
medium gcnn:GraphConv                     0.133785  0.200214
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  1.932990  3.195202
       ml-competitor:lambdamart_khalil    0.605084  0.509906
small  gcnn:GraphConv                     0.202993  0.364124
       gcnn:Lodi                          1.000000  1.000000
       ml-competitor:extratrees_gcnn_agg  0.553939  0.728804
       ml-competitor:lambdamart_khalil    0.450425  0.409704

# 2. Std variances (per instance)

In [14]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','policy','instance']).std() / df_gcnns.groupby(['type','policy','instance']).mean()

In [15]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','policy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   policy                                               
big    gcnn:GraphConv                     0.364426  0.302025
       gcnn:Lodi                          0.451601  0.241620
       ml-competitor:extratrees_gcnn_agg  0.093133  0.023651
       ml-competitor:lambdamart_khalil    0.297003  0.090318
medium gcnn:GraphConv                     0.949034  0.814071
       gcnn:Lodi                          0.832033  0.820532
       ml-competitor:extratrees_gcnn_agg  0.177377  0.115656
       ml-competitor:lambdamart_khalil    0.857202  0.771794
small  gcnn:GraphConv                     0.536642  0.161312
       gcnn:Lodi                          1.604312  1.029292
       ml-competitor:extratrees_gcnn_agg  0.710777  0.368205
       ml-competitor:lambdamart_khalil    1.089412  0.364836

In [16]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'gcnn:Lodi'),metric]
        for policy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,policy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,policy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,policy),(metric,)] = [t_stats, p_value]

In [17]:
ttest_res

nnodes               stime  \
                                           t_stats   p_value   t_stats   
type   policy                                                            
big    gcnn:GraphConv                    -1.911683  0.057361 -2.494493   
       gcnn:Lodi                               0.0       1.0       0.0   
       ml-competitor:extratrees_gcnn_agg  -8.36138       0.0  4.255608   
       ml-competitor:lambdamart_khalil   -0.159859  0.873155  1.545771   
medium gcnn:GraphConv                    -7.717763       0.0 -7.750952   
       gcnn:Lodi                               0.0       1.0       0.0   
       ml-competitor:extratrees_gcnn_agg -1.537647  0.125731  5.549974   
       ml-competitor:lambdamart_khalil   -1.310691  0.191481 -4.362353   
small  gcnn:GraphConv                     -6.21847       0.0 -9.619463   
       gcnn:Lodi                               0.0       1.0       0.0   
       ml-competitor:extratrees_gcnn_agg -5.076601  0.000001 -7.460072   
       ml-competitor:lambdamart_khalil   -5.980931       0.0 -9.511505   

                                                    
                                           p_value  
type   policy                                       
big    gcnn:GraphConv                     0.013432  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg  0.000032  
       ml-competitor:lambdamart_khalil    0.123757  
medium gcnn:GraphConv                          0.0  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg       0.0  
       ml-competitor:lambdamart_khalil    0.000021  
small  gcnn:GraphConv                          0.0  
       gcnn:Lodi                               1.0  
       ml-competitor:extratrees_gcnn_agg       0.0  
       ml-competitor:lambdamart_khalil         0.0

In [18]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type   instance                                             seed
big    data/instances/indset/transfer_1500_4/instance_1.lp  0       405.0
                                                            1       401.0
                                                            2       407.0
                                                            3       403.0
                                                            4       409.0
                                                                    ...  
small  data/instances/indset/transfer_500_4/instance_9.lp   0       730.0
                                                            1       731.0
                                                            2       732.0
                                                            3        88.0
                                                            4       734.0
Name: stime, Length: 300, dtype: float64

In [19]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [20]:
len(winner_idx)

300

In [21]:
dfgcnns_wins = df_gcnns.groupby(['type', 'policy'])['wins'].sum()
dfgcnns_wins

type    policy                           
big     gcnn:GraphConv                       50
        gcnn:Lodi                            27
        ml-competitor:extratrees_gcnn_agg     9
        ml-competitor:lambdamart_khalil      14
medium  gcnn:GraphConv                       66
        gcnn:Lodi                            24
        ml-competitor:extratrees_gcnn_agg     0
        ml-competitor:lambdamart_khalil      10
small   gcnn:GraphConv                       20
        gcnn:Lodi                            19
        ml-competitor:extratrees_gcnn_agg     6
        ml-competitor:lambdamart_khalil      55
Name: wins, dtype: int64

In [22]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

In [23]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std  \
                                            nnodes     stime    nnodes   
type   policy                                                            
big    gcnn:GraphConv                     0.761373  0.789125  0.364426   
       gcnn:Lodi                          1.000000  1.000000  0.451601   
       ml-competitor:extratrees_gcnn_agg  0.756533  1.290639  0.093133   
       ml-competitor:lambdamart_khalil    1.028079  1.062454  0.297003   
medium gcnn:GraphConv                     0.133785  0.200214  0.949034   
       gcnn:Lodi                          1.000000  1.000000  0.832033   
       ml-competitor:extratrees_gcnn_agg  1.932990  3.195202  0.177377   
       ml-competitor:lambdamart_khalil    0.605084  0.509906  0.857202   
small  gcnn:GraphConv                     0.202993  0.364124  0.536642   
       gcnn:Lodi                          1.000000  1.000000  1.604312   
       ml-competitor:extratrees_gcnn_agg  0.553939  0.728804  0.710777   
       ml-competitor:lambdamart_khalil    0.450425  0.409704  1.089412   

                                                   wins  
                                             stime wins  
type   policy                                            
big    gcnn:GraphConv                     0.302025   50  
       gcnn:Lodi                          0.241620   27  
       ml-competitor:extratrees_gcnn_agg  0.023651    9  
       ml-competitor:lambdamart_khalil    0.090318   14  
medium gcnn:GraphConv                     0.814071   66  
       gcnn:Lodi                          0.820532   24  
       ml-competitor:extratrees_gcnn_agg  0.115656    0  
       ml-competitor:lambdamart_khalil    0.771794   10  
small  gcnn:GraphConv                     0.161312   20  
       gcnn:Lodi                          1.029292   19  
       ml-competitor:extratrees_gcnn_agg  0.368205    6  
       ml-competitor:lambdamart_khalil    0.364836   55

In [24]:
output_idx = pd.MultiIndex.from_product((problemSizes, models), names=['type', 'policy'])
output_idx

MultiIndex([( 'small',                         'gcnn:Lodi'),
            ( 'small', 'ml-competitor:extratrees_gcnn_agg'),
            ( 'small',   'ml-competitor:lambdamart_khalil'),
            ( 'small',                    'gcnn:GraphConv'),
            ('medium',                         'gcnn:Lodi'),
            ('medium', 'ml-competitor:extratrees_gcnn_agg'),
            ('medium',   'ml-competitor:lambdamart_khalil'),
            ('medium',                    'gcnn:GraphConv'),
            (   'big',                         'gcnn:Lodi'),
            (   'big', 'ml-competitor:extratrees_gcnn_agg'),
            (   'big',   'ml-competitor:lambdamart_khalil'),
            (   'big',                    'gcnn:GraphConv')],
           names=['type', 'policy'])

In [25]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [26]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   policy                                                             
small  gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN
medium gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN
big    gcnn:Lodi                            NaN   NaN  NaN             NaN
       ml-competitor:extratrees_gcnn_agg    NaN   NaN  NaN             NaN
       ml-competitor:lambdamart_khalil      NaN   NaN  NaN             NaN
       gcnn:GraphConv                       NaN   NaN  NaN             NaN

In [27]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [29]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [30]:
out_df1

nnodes  \
type   policy                                                 
small  gcnn:Lodi                          1.0000r ± 160.43%   
       ml-competitor:extratrees_gcnn_agg   0.5539r ± 71.08%   
       ml-competitor:lambdamart_khalil    0.4504r ± 108.94%   
       gcnn:GraphConv                      0.2030r ± 53.66%   
medium gcnn:Lodi                           1.0000r ± 83.20%   
       ml-competitor:extratrees_gcnn_agg   1.9330r ± 17.74%   
       ml-competitor:lambdamart_khalil     0.6051r ± 85.72%   
       gcnn:GraphConv                      0.1338r ± 94.90%   
big    gcnn:Lodi                           1.0000r ± 45.16%   
       ml-competitor:extratrees_gcnn_agg    0.7565r ± 9.31%   
       ml-competitor:lambdamart_khalil     1.0281r ± 29.70%   
       gcnn:GraphConv                      0.7614r ± 36.44%   

                                                      stime  wins  \
type   policy                                                       
small  gcnn:Lodi                          1.0000r ± 102.93%    19   
       ml-competitor:extratrees_gcnn_agg   0.7288r ± 36.82%     6   
       ml-competitor:lambdamart_khalil     0.4097r ± 36.48%    55   
       gcnn:GraphConv                      0.3641r ± 16.13%    20   
medium gcnn:Lodi                           1.0000r ± 82.05%    24   
       ml-competitor:extratrees_gcnn_agg   3.1952r ± 11.57%     0   
       ml-competitor:lambdamart_khalil     0.5099r ± 77.18%    10   
       gcnn:GraphConv                      0.2002r ± 81.41%    66   
big    gcnn:Lodi                           1.0000r ± 24.16%    27   
       ml-competitor:extratrees_gcnn_agg    1.2906r ± 2.37%     9   
       ml-competitor:lambdamart_khalil      1.0625r ± 9.03%    14   
       gcnn:GraphConv                      0.7891r ± 30.20%    50   

                                         t-stats (p-val)  
type   policy                                             
small  gcnn:Lodi                            0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg   -7.46(0.0000)  
       ml-competitor:lambdamart_khalil     -9.51(0.0000)  
       gcnn:GraphConv                      -9.62(0.0000)  
medium gcnn:Lodi                            0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg    5.55(0.0000)  
       ml-competitor:lambdamart_khalil     -4.36(0.0000)  
       gcnn:GraphConv                      -7.75(0.0000)  
big    gcnn:Lodi                            0.00(1.0000)  
       ml-competitor:extratrees_gcnn_agg    4.26(0.0000)  
       ml-competitor:lambdamart_khalil      1.55(0.1238)  
       gcnn:GraphConv                      -2.49(0.0134)